In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn as nn



In [35]:
train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp2.csv' ,delimiter=',')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp2.csv' ,delimiter=',')



In [36]:
display(train.head())

,train_id,item_condition_id,price,shipping,vec0,vec1,vec2,vec3,vec4,vec5,...,cat_Electronics,cat_Fashion,cat_Food and Beverage,cat_Furniture and Decor,cat_Health and Wellness,cat_Household Items,cat_Media,cat_Office Equipment,cat_Pet Care,cat_Toys and Hobbies
0,0,3,10.0,1,-0.007633,0.013877,0.001542,0.002168,-0.002364,-0.014765,...,0,1,0,0,0,0,0,0,0,0
1,1,3,52.0,0,0.013427,-0.005142,0.006128,0.001119,-0.010722,0.013957,...,1,0,0,0,0,0,0,0,0,0
2,2,1,10.0,1,0.001426,-0.006458,0.002439,0.008610,0.012070,-0.011089,...,0,1,0,0,0,0,0,0,0,0
3,3,1,35.0,1,0.005893,-0.008465,0.012312,-0.003756,0.010533,-0.000997,...,0,0,0,1,0,0,0,0,0,0
4,4,1,44.0,0,0.008761,0.012107,-0.013877,-0.008480,-0.004083,-0.001077,...,0,1,0,0,0,0,0,0,0,0


In [37]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [38]:
# Separate features and target
X = train.drop(columns=['price'])
y = train['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
display(X_test)

,train_id,item_condition_id,shipping,vec0,vec1,vec2,vec3,vec4,vec5,vec6,...,cat_Electronics,cat_Fashion,cat_Food and Beverage,cat_Furniture and Decor,cat_Health and Wellness,cat_Household Items,cat_Media,cat_Office Equipment,cat_Pet Care,cat_Toys and Hobbies
639825,640169,4,1,-0.007518,-0.005747,0.006878,0.009840,-0.004053,-0.005023,-0.008929,...,0,1,0,0,0,0,0,0,0,0
506709,506976,1,0,-0.008284,-0.012492,0.000325,0.012539,0.003697,0.003601,-0.014611,...,0,1,0,0,0,0,0,0,0,0
1391997,1392815,3,1,-0.007041,0.012647,0.009331,0.010502,0.008051,0.000927,0.005459,...,0,1,0,0,0,0,0,0,0,0
805573,806019,3,1,0.014300,-0.007132,0.000750,-0.008893,0.007539,-0.000409,0.009767,...,0,0,0,1,0,0,0,0,0,0
669893,670253,2,1,-0.001054,0.004260,0.003415,-0.013736,0.013255,0.008476,0.003266,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997303,997867,1,1,0.014646,-0.001663,0.015255,0.003389,-0.007724,0.001575,-0.006661,...,0,0,0,0,0,0,0,0,0,0
1064756,1065354,2,0,-0.013161,0.011698,-0.001355,-0.008432,-0.004804,0.013311,-0.007393,...,0,0,0,0,0,0,0,0,0,0
346499,346660,1,0,-0.004227,-0.015071,0.011770,-0.007321,0.012993,-0.005967,0.003857,...,0,1,0,0,0,0,0,0,0,0
1091705,1092325,2,1,-0.012440,-0.001498,-0.005188,0.014648,0.003431,-0.003886,-0.004036,...,0,0,0,1,0,0,0,0,0,0


create data for pytorch

In [40]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [41]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# Reshape y tensors to have the correct shape (n_samples, 1)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


In [42]:

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Model Building

In [53]:
class CNNModel(nn.Module):
    def __init__(self, num_features):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * num_features, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = self.conv1(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        return x

class RegularizedModel(nn.Module):
    def __init__(self, num_features):
        super(RegularizedModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class ComplexModel(nn.Module):
    def __init__(self, num_features):
        super(ComplexModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x
class EnhancedModel(nn.Module):
    def __init__(self, num_features):
        super(EnhancedModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5)
        ).to(device=device)
        self.layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.4)
        ).to(device=device)
        self.layer3 = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3)
        ).to(device=device)
        self.final_layer = nn.Linear(64, 1).to(device=device)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.final_layer(x)
        return x



In [46]:
def rmsle(y_true, y_pred):
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))

def train_model(model, train_loader, test_loader, num_epochs=10):
    criterion = nn.MSELoss()  # Mean Squared Error Loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            total_loss = 0
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs = model(inputs)
                total_loss += rmsle(targets, outputs).item()

        print(f'Epoch {epoch+1}, RMSLE: {total_loss / len(test_loader)}')

# Example usage
num_features = X_train.shape[1]
model = CNNModel(num_features).to(device)
train_model(model, train_loader, test_loader)
#Epoch 4, RMSLE: 0.7835791014590188


Epoch 1, RMSLE: 0.7709122869165792
Epoch 2, RMSLE: 0.7603577460101809
Epoch 3, RMSLE: 0.7743236570681591
Epoch 4, RMSLE: 0.7835791014590188
Epoch 5, RMSLE: 0.7580822768131051
Epoch 6, RMSLE: 0.7721207110214069
Epoch 7, RMSLE: 0.7634894715944239
Epoch 8, RMSLE: 0.7784918436631351
Epoch 9, RMSLE: 0.7687526159398732
Epoch 10, RMSLE: 0.7700883977435315


In [47]:
model2 = RegularizedModel(num_features).to(device)
train_model(model2, train_loader, test_loader)
#Epoch 6, RMSLE: 0.7613243316759741


Epoch 1, RMSLE: 0.7533622819168525
Epoch 2, RMSLE: 0.7580342817399188
Epoch 3, RMSLE: 0.7562796770818443
Epoch 4, RMSLE: 0.7514503072055156
Epoch 5, RMSLE: 0.7446639232387349
Epoch 6, RMSLE: 0.7613243316759741
Epoch 7, RMSLE: 0.7581775750513597
Epoch 8, RMSLE: 0.7555380024319976
Epoch 9, RMSLE: 0.7530308805488145
Epoch 10, RMSLE: 0.7530687315318323


In [48]:
model3 = ComplexModel(num_features).to(device)
train_model(model3, train_loader, test_loader)
#Epoch 5, RMSLE: 0.7639840310244395


Epoch 1, RMSLE: 0.7620540489467785
Epoch 2, RMSLE: 0.7430975818397211
Epoch 3, RMSLE: 0.7468285496125234
Epoch 4, RMSLE: 0.7493711375078149
Epoch 5, RMSLE: 0.7639840310244395
Epoch 6, RMSLE: 0.7522793242571498
Epoch 7, RMSLE: 0.7434691373999704
Epoch 8, RMSLE: 0.7570093201396533
Epoch 9, RMSLE: 0.7547818548193865
Epoch 10, RMSLE: 0.7553681071853309


In [54]:
model4 = EnhancedModel(num_features).to(device)
train_model(model4, train_loader, test_loader)
# Epoch 8, RMSLE: 0.7595992399147534


Epoch 1, RMSLE: 0.7525199949548301
Epoch 2, RMSLE: 0.7594808512669973
Epoch 3, RMSLE: 0.743288447367298
Epoch 4, RMSLE: 0.7398904664834101
Epoch 5, RMSLE: 0.7413829566206589
Epoch 6, RMSLE: 0.7508822263921731
Epoch 7, RMSLE: 0.7426449539877122
Epoch 8, RMSLE: 0.7595992399147534
Epoch 9, RMSLE: 0.74301114137166
Epoch 10, RMSLE: 0.7490666871116167
